In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv("tmdb_100_credits.csv")

In [3]:
df.drop(['Unnamed: 0'], axis=1,inplace=True)

In [4]:
df.head()

,movie_id,title,cast,crew,lead_actor_name,URL,WikiData,Birthday,Full Name,Birthplace,IMDB_reviews_url,other_reviews
0,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Sam Worthington,https://en.wikipedia.org/wiki/Sam_Worthington,"{'Birthday': '1976-08-02', 'Nickname': 'Samuel...",1976-08-02,Samuel Henry John Worthington,"Godalming, Surrey, England, UK",['https://www.imdb.com/title/tt0499549/reviews...,[{'Review': 'What if the director of the highe...
1,285,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Johnny Depp,https://en.wikipedia.org/wiki/Johnny_Depp,"{'Birthday': '1963-06-09', 'Nickname': 'John C...",1963-06-09,John Christopher Depp II,"Owensboro, Kentucky, U.S.",['https://www.imdb.com/title/tt0449088/reviews'],[{'Review': 'Jack Sparrow and his swashbucklin...
2,206647,Spectre,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Daniel Craig,https://en.wikipedia.org/wiki/Daniel_Craig,"{'Birthday': '1968-03-02', 'Nickname': 'Daniel...",1968-03-02,Daniel Wroughton Craig,"Chester, Cheshire, England",['https://www.imdb.com/title/tt2379713/reviews...,"[{'Review': 'If there is such a thing as ""Jame..."
3,49026,The Dark Knight Rises,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Christian Bale,https://en.wikipedia.org/wiki/Christian_Bale,"{'Birthday': '1974-01-30', 'Nickname': 'Christ...",1974-01-30,Christian Charles Philip Bale,"Haverfordwest, Pembrokeshire, Wales","No, it wasn't found","[{'Review': 'With their Batman trilogy, Nolan ..."
4,49529,John Carter,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",Taylor Kitsch,https://en.wikipedia.org/wiki/Taylor_Kitsch,"{'Birthday': 'None', 'Nickname': 'None', 'Birt...",None,None,None,['https://www.imdb.com/title/tt0401729/reviews'],"[{'Review': ""Let's start with the obvious: Kit..."


In [5]:
# extraxt the title number out of the IMDB_reviews_url column for each movie 
# so that we can get the IMDB url for criticreviews.
def get_critic_review_url(row):
    if row == "No, it wasn't found" or row == "Run into erro":
        return "None"
    else:
        title_number = row.split(",")[0].rsplit('/')[4]
        critic_review_url = f"https://www.imdb.com/title/{title_number}/criticreviews"
        return critic_review_url

In [6]:
df['IMDB_critic_reviews_url'] = df["IMDB_reviews_url"].map(get_critic_review_url)
df.head(2)

,movie_id,title,cast,crew,lead_actor_name,URL,WikiData,Birthday,Full Name,Birthplace,IMDB_reviews_url,other_reviews,IMDB_critic_reviews_url
0,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Sam Worthington,https://en.wikipedia.org/wiki/Sam_Worthington,"{'Birthday': '1976-08-02', 'Nickname': 'Samuel...",1976-08-02,Samuel Henry John Worthington,"Godalming, Surrey, England, UK",['https://www.imdb.com/title/tt0499549/reviews...,[{'Review': 'What if the director of the highe...,https://www.imdb.com/title/tt0499549/criticrev...
1,285,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Johnny Depp,https://en.wikipedia.org/wiki/Johnny_Depp,"{'Birthday': '1963-06-09', 'Nickname': 'John C...",1963-06-09,John Christopher Depp II,"Owensboro, Kentucky, U.S.",['https://www.imdb.com/title/tt0449088/reviews'],[{'Review': 'Jack Sparrow and his swashbucklin...,https://www.imdb.com/title/tt0449088/criticrev...


In [7]:
# we have 14 rows that do not have IMDB_reviews_url to begin with.
# That is becasue splinter failed to return google search result in part I.
df[df['IMDB_critic_reviews_url'] == "None"]['IMDB_critic_reviews_url'].count()

14

In [8]:
# Write a function to scrape critic reviews from IMDB
def get_IMDB_critic_reviews(row):
    url = row['IMDB_critic_reviews_url']
    if url == "None":
        return "None"
    else:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content, "html.parser")
        soup_objects = soup.find_all("td",{"class","review"})
        result = []
        for x in soup_objects:
            review = x.find("div",{"class","summary"}).get_text()
            publisher = x.find('span').get_text()
            author = x.find("span",{"itemprop":"author"}).get_text()
            result.append({"Movie_ID":row['movie_id'], "Publisher": publisher, "Author": author,"Review": review})
        return result

In [9]:
df["IMDB_critic_reviews"] = df.apply(get_IMDB_critic_reviews, axis=1)

In [10]:
df.index

RangeIndex(start=0, stop=100, step=1)

In [11]:
IMDB_critic_review_list = []
for x in range(0,100):
    try:
        IMDB_critic_review_list = IMDB_critic_review_list + df["IMDB_critic_reviews"][x]
    except TypeError:
        pass

In [12]:
IMDB_critic_reviews = pd.DataFrame.from_dict(IMDB_critic_review_list)
IMDB_critic_reviews.head()

,Author,Movie_ID,Publisher,Review
0,Kirk Honeycutt,19995,The Hollywood Reporter,"A fully believable, flesh-and-blood (albeit n..."
1,Chris Hewitt (1),19995,Empire,"It's been twelve years since ""Titanic,"" but t..."
2,Roger Ebert,19995,Chicago Sun-Times,"Once again, [Cameron] has silenced the doubte..."
3,Todd McCarthy,19995,Variety,"Avatar is all-enveloping and transporting, wi..."
4,Richard Corliss,19995,Time,Embrace the movie -- surely the most vivid an...


In [13]:
look_up = dict(zip(df['movie_id'],df['IMDB_critic_reviews_url']))

In [14]:
IMDB_critic_reviews['IMDB_critic_reviews_url'] = IMDB_critic_reviews['Movie_ID'].map(lambda x: look_up.get(x))
IMDB_critic_reviews.head()

,Author,Movie_ID,Publisher,Review,IMDB_critic_reviews_url
0,Kirk Honeycutt,19995,The Hollywood Reporter,"A fully believable, flesh-and-blood (albeit n...",https://www.imdb.com/title/tt0499549/criticrev...
1,Chris Hewitt (1),19995,Empire,"It's been twelve years since ""Titanic,"" but t...",https://www.imdb.com/title/tt0499549/criticrev...
2,Roger Ebert,19995,Chicago Sun-Times,"Once again, [Cameron] has silenced the doubte...",https://www.imdb.com/title/tt0499549/criticrev...
3,Todd McCarthy,19995,Variety,"Avatar is all-enveloping and transporting, wi...",https://www.imdb.com/title/tt0499549/criticrev...
4,Richard Corliss,19995,Time,Embrace the movie -- surely the most vivid an...,https://www.imdb.com/title/tt0499549/criticrev...


In [16]:
import ast
def add_movieid (row):
    try:
        reviews = ast.literal_eval(row['other_reviews'])
        for r in reviews:
            r['movie_id'] = row['movie_id']
        return reviews
    except SyntaxError:
        pass
        

df['other_reviews_with_movieid'] = df.apply(add_movieid,axis=1)

In [17]:
df.head(2)

,movie_id,title,cast,crew,lead_actor_name,URL,WikiData,Birthday,Full Name,Birthplace,IMDB_reviews_url,other_reviews,IMDB_critic_reviews_url,IMDB_critic_reviews,other_reviews_with_movieid
0,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Sam Worthington,https://en.wikipedia.org/wiki/Sam_Worthington,"{'Birthday': '1976-08-02', 'Nickname': 'Samuel...",1976-08-02,Samuel Henry John Worthington,"Godalming, Surrey, England, UK",['https://www.imdb.com/title/tt0499549/reviews...,[{'Review': 'What if the director of the highe...,https://www.imdb.com/title/tt0499549/criticrev...,"[{'Movie_ID': 19995, 'Publisher': 'The Hollywo...",[{'Review': 'What if the director of the highe...
1,285,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Johnny Depp,https://en.wikipedia.org/wiki/Johnny_Depp,"{'Birthday': '1963-06-09', 'Nickname': 'John C...",1963-06-09,John Christopher Depp II,"Owensboro, Kentucky, U.S.",['https://www.imdb.com/title/tt0449088/reviews'],[{'Review': 'Jack Sparrow and his swashbucklin...,https://www.imdb.com/title/tt0449088/criticrev...,"[{'Movie_ID': 285, 'Publisher': 'Chicago Tribu...",[{'Review': 'Jack Sparrow and his swashbucklin...


In [18]:
other_reviews_list = []
for x in range(0,100):
    try:
        other_reviews_list = other_reviews_list + df['other_reviews_with_movieid'][x] 
    except TypeError:
        pass
other_reviews_list[0]

{'Review': 'What if the director of the highest-grossing movie ever made (Titanic) spent a rumored $500 million on a spectacular futuristic sci-fi epic and no one other than hardcore fanboys went to see it? Full review',
 'Source': 'https://www.commonsensemedia.org/movie-reviews/avatar',
 'movie_id': 19995}

In [19]:
Other_Source_Reviews = pd.DataFrame.from_dict(other_reviews_list)
Other_Source_Reviews.head()

,Review,Source,movie_id
0,What if the director of the highest-grossing m...,https://www.commonsensemedia.org/movie-reviews...,19995
1,"Worth watching for fans, completists and anyon...",https://www.empireonline.com/movies/avatar-spe...,19995
2,But Avatar is no Hollywood wankfest. It extend...,https://www.rollingstone.com/movies/movie-revi...,19995
3,"Fifteen years in the making, James Cameron's l...",https://www.pluggedin.com/movie-reviews/avatar/,19995
4,Does James Cameron's Avatar movie live up to a...,https://screenrant.com/avatar-reviews,19995


In [25]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine("sqlite:///movieactors.sqlite")
import sqlite3

conn = sqlite3.connect('movieactors.sqlite')

In [26]:
IMDB_critic_reviews.to_sql('IMDB critic reviews', conn, if_exists='replace', index=False)

In [27]:
Other_Source_Reviews.to_sql('Other source reviews', conn, if_exists='replace', index=False)

In [29]:
df_movie = pd.read_csv('tmdb_5000_movies.csv')

In [32]:
df_movie.shape

(4803, 20)

In [37]:
df_movie.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [40]:
df_to_save = df_movie[['budget','homepage','original_language',\
                            'original_title','overview','popularity','release_date',\
                           'revenue','runtime','status','tagline','title','vote_average','vote_count']]

In [44]:
df_to_save.to_sql('TMDB 5000 Movies', conn, if_exists='replace', index=False)